In [1]:
# mypoll/ORM_Exam.ipynb

# Jupyter Lab에서 Django shell을 실행하기 위한 설정
import os
import django
# 환경변수로 config/settings.py의 위치를 설정
os.environ["DJANGO_SETTINGS_MODULE"] = "config.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [3]:
# 조회 테스트
from polls.models import Question, Choice

q = Question.objects.all()
q

<QuerySet [<Question: 1. 좋아하는 색깔은 무엇입니까?>, <Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔을 고르세요>, <Question: 4. 여행가고 싶은 나라는?>]>

# 조회
- ModelClass.object -> Model Manager를 반환.
- ModelManager: SQL작업을 할 수 있는 메소드들을 제공하는 객체

## 조회 메소드
- `all()`: 전체 조회
- `filter()`, `exclude()`: 조건으로 조회(where절)
- `get()`: 조회결과가 하나인 조건으로 조회(PK로 조회)

## 조회결과
- `QuerySet` 객체: 조회결과가 여러 개일 때 QuerySet에 모아서 반환.
    - 조회 결과를 바탕으로 추가 DB 작업을 진행할 수 있다.
    - 개별 데이터는 Model 객체에 담아서 반환
- Model 객체: 조회 결과가 하나 (`get()`)일 때

In [2]:
from polls.models import Question, Choice

In [3]:
model_manager = Question.objects
type(model_manager)

django.db.models.manager.Manager

In [5]:
result = model_manager.all()
print("조회한 데이터 개수:", len(result))
print("all()로 실행된 SQL문을 조회")
print(result.query)

조회한 데이터 개수: 4
all()로 실행된 SQL문을 조회
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [6]:
print(type(result))
result

<class 'django.db.models.query.QuerySet'>


<QuerySet [<Question: 1. 좋아하는 색깔은 무엇입니까?>, <Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔을 고르세요>, <Question: 4. 여행가고 싶은 나라는?>]>

In [31]:
# QuerySet -> Iterable
q = result[1]
print(q.id, q.pk)
print(q.question_text, q.pub_date)
print(type(q.pk), type(q.question_text), type(q.pub_date))

2 2
좋아하는 동물은 무엇입니까? 2025-07-07 05:17:27.658555+00:00
<class 'int'> <class 'str'> <class 'datetime.datetime'>


In [33]:
# QuerySet - 첫 번째, 마지막 index값 조회
q_s = result.first()
q_e = result.last()
q_s.pk, q_e.pk

(1, 4)

In [36]:
# slicing -> 결과: list
s_result = result[1:3] # 음수 인덱싱은 지원하지 않는다.
print(type(s_result))
s_result

<class 'list'>


[<Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔을 고르세요>]

In [44]:
# QuerySet을 이용해서 정렬 (order by)
#     - QS.orderby("기준Field명"): ASC, QS.orderby("-기준Field명"): DESC
result.order_by("question_text")

<QuerySet [<Question: 3. 싫어하는 색깔을 고르세요>, <Question: 4. 여행가고 싶은 나라는?>, <Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 1. 좋아하는 색깔은 무엇입니까?>]>

In [46]:
# Question.objects.all().order_by("q_t")
# select *
# from Question
# order by question_text desc

In [55]:
# Choice에 모든 데이터를 조회 -> question 컬럼 기준으로 정렬
results = Choice.objects.all().order_by("choice_text", "-votes")
for c in results:
    print(c.pk, c.choice_text, c.votes) 

2 노란색 0
6 보라색 5
4 보라색 0
5 빨간색 3
3 주황색 0
1 파란색 7


In [56]:
print(results.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" ASC, "polls_choice"."votes" DESC


### Where 절
- filter()
    - 조회 조건이 True인 행들을 조회 -> QuerySet을 반환
- exclude()
    - 조회 조건이 False인 행들을 조회 -> QuerySet을 반환
- get()
    - 조회 조건이 True인 행이 1개일 때 조회 -> Model에 결과를 담아서 반환
    - 조회 결과가 2행 이상 or 없을 경우 Exception 발생
- **조회조건**
    - `Field이름__비교연산자 = 비교할 값`

In [64]:
# pk 조회 -> 결과 1행 | 0행
result = Question.objects.get(pk=1) # where id = 1. 조회 결과가 1개
result = Question.objects.filter(pk=1) # where id = 1. 조회 결과가 0개 이상
result = Question.objects.exclude(pk=1) # where id != 1.
print(type(result))
print(result)

<class 'django.db.models.query.QuerySet'>
<QuerySet [<Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔을 고르세요>, <Question: 4. 여행가고 싶은 나라는?>]>


In [70]:
# 비교 연산
result = Choice.objects.filter(pk__lt=5) # where pk < 5
result = Choice.objects.filter(pk__lte=5) # where pk <= 5
result = Choice.objects.filter(pk__gt=5) # where pk > 5
result = Choice.objects.filter(pk__gte=5) # where pk >= 5
result = Choice.objects.filter(pk=5) # where pk = 5

for r in result:
    print(r)

5. 빨간색


In [80]:
# 문자열 부분일치 - like (xxx를 포함, xxx로 시작, xxx로 끝)
result = Question.objects.filter(question_text__contains='색깔') # question_text like '%색깔%'
result = Question.objects.filter(question_text__endswith='고르세요') # question_text like '%고르세요'
result = Question.objects.filter(question_text__startswith='싫어하는') # question_text like '싫어하는%'

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE 싫어하는% ESCAPE '\'
3. 싫어하는 색깔을 고르세요


In [81]:
# in 연산
result = Choice.objects.filter(pk__in=[1,3,10]) # pk in (1, 3, 10)
result = Choice.objects.exclude(pk__in=[1,3,10]) # pk not in (1, 3, 10)

print(result.query)
for r in result:
    print(r)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" IN (1, 3, 10))
2. 노란색
4. 보라색
5. 빨간색
6. 보라색


In [85]:
# between
result = Choice.objects.filter(pk__range=[4, 8]) # pk between 4 and 8
result = Choice.objects.exclude(pk__range=[4, 8]) # pk between 4 and 8

print(result.query)
for r in result:
    print(r)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" BETWEEN 4 AND 8)
1. 파란색
2. 노란색
3. 주황색


#### where절의 and, or
- `and`: 조건들을 나열
- 각 조건을 `Q()`함수에 넣고 `|`로 연결

In [7]:
from django.db.models import Q

result = Question.objects.filter(
    ~Q(question_text__endswith='고르세요') | Q(pk__gte=3)
    )

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE (NOT ("polls_question"."question_text" LIKE %고르세요 ESCAPE '\') OR "polls_question"."id" >= 3)
1. 좋아하는 색깔은 무엇입니까?
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔을 고르세요
4. 여행가고 싶은 나라는?


In [88]:
Q(question_text__endswith='고르세요') | Q(pk__gte=3)

<Q: (OR: ('question_text__endswith', '고르세요'), ('pk__gte', 3))>

### 조회할 컬럼들을 선택.
- `values(Field명, ...)`
- 개별 조회 결과를 Dictionary로 반환.

In [95]:
result = Question.objects.all().values("pk","question_text")
result = Choice.objects.filter(pk__lt=5).values('pk','votes')
# select pk, votes from choice where pk < 5
print(result.query)
result
# for r in result:
#     print(type(r), r['pk'], r['question_text'])

SELECT "polls_choice"."id" AS "pk", "polls_choice"."votes" AS "votes" FROM "polls_choice" WHERE "polls_choice"."id" < 5


<QuerySet [{'pk': 1, 'votes': 7}, {'pk': 2, 'votes': 0}, {'pk': 3, 'votes': 0}, {'pk': 4, 'votes': 0}]>

### 집계함수
- `aggregate(집계함수(집계기준 field명), 집계함수(field명), ...)`
    - `select avg(salary), count(comm_pct), max(salary) from emp`

**groupby**


In [97]:
from django.db.models import (
    Count, # 값의 개수(null 제외한)
    Sum,   # 합계
    Avg,   # 평균
    Min,   # 최솟값
    Max,   # 최댓값
    StdDev, # 표준편차
    Variance # 분산
)

In [100]:
result = Choice.objects.aggregate(
    Count("votes"),
    Avg("votes"),
    Max("pk"),
    Min("pk"),
    Sum("votes"),
    StdDev("votes"),
    Variance("votes")
)

print(result)
# 반환: dictionary
# default key: field명__집계명

{'votes__count': 6, 'votes__avg': 2.5, 'pk__max': 6, 'pk__min': 1, 'votes__sum': 15, 'votes__stddev': 2.753785273643051, 'votes__variance': 7.583333333333333}


In [101]:
result = Choice.objects.aggregate(
    cnt=Count("votes"),
    min=Min("votes"),
    max=Max("votes")
)
result

{'cnt': 6, 'min': 0, 'max': 7}

In [103]:
# 집계 결과를 연산
# 변수명 = (집계함수 - 집계함수)
Choice.objects.aggregate(min_max_diff = (Max("pk") - Min("pk")))

{'min_max_diff': 5}

In [107]:
# groupby + 집계
result = Choice.objects.values("question").annotate(
    min=Min("votes"),
    max=Max("votes")
)
result

<QuerySet [{'question': 1, 'min': 0, 'max': 7}]>

### JOIN
- 자식 테이블(모델) 기준으로 부모 테이블(모델) 데이터 조회
- `자식모델객체.FK_Field`

In [8]:
# Choice(자식) --> Question(부모)
c1 = Choice.objects.get(pk=1)
c1.pk, c1.id, c1.choice_text, c1.votes, c1.question

(1, 1, '파란색', 7, <Question: 1. 좋아하는 색깔은 무엇입니까?>)

In [9]:
# c1이 참조하는 question의 정보
c1.question.pk, c1.question.question_text, c1.question.pub_date

(1,
 '좋아하는 색깔은 무엇입니까?',
 datetime.datetime(2025, 7, 7, 5, 17, 1, 655423, tzinfo=datetime.timezone.utc))

In [16]:
result_list = Choice.objects.filter(pk__lte=5)
# 조회한 Choice들의 질문 - 보기
for result in result_list:
    print(f"질문:{result.question.question_text}, 버디:{result.choice_text}")

질문:좋아하는 색깔은 무엇입니까?, 버디:파란색
질문:좋아하는 색깔은 무엇입니까?, 버디:노란색
질문:좋아하는 색깔은 무엇입니까?, 버디:주황색
질문:좋아하는 색깔은 무엇입니까?, 버디:보라색
질문:좋아하는 색깔은 무엇입니까?, 버디:빨간색


- 부모테이블(모델) 기준으로 자식테이블의 데이터(모델)를 조회
    - `부모모델객체.자식모델클래스이름(소문자)_set`를 통해서 부모객체를 참조하는 자식 데이터들을 조회할 수 있다.

In [17]:
q1 = Question.objects.get(pk=1)
q1
print(q1.pk, q1.question_text, q1.pub_date)

1 좋아하는 색깔은 무엇입니까? 2025-07-07 05:17:01.655423+00:00


In [19]:
# Related Manager -> 부모객체(q1)와 관련 있는 자식의 데이터들 안에서만 조회할 수 있는 Model Manager
q1.choice_set

choice_list = q1.choice_set.all()
choice_list

<QuerySet [<Choice: 1. 파란색>, <Choice: 2. 노란색>, <Choice: 3. 주황색>, <Choice: 4. 보라색>, <Choice: 5. 빨간색>, <Choice: 6. 보라색>]>

In [20]:
print("질문:", q1.question_text)
print("버디:")
for c in choice_list:
    print(f"{c.pk}. {c.choice_text}, {c.votes}")

질문: 좋아하는 색깔은 무엇입니까?
버디:
1. 파란색, 7
2. 노란색, 0
3. 주황색, 0
4. 보라색, 0
5. 빨간색, 3
6. 보라색, 5


In [22]:
# 전체 질문을 조회하고 그것에 대해서 위 형식으로 출력.
q_list = Question.objects.all()
for q in q_list:
    print(f"{q.pk}. {q.question_text}")
    c_list = q.choice_set.all()
    for idx, c in enumerate(c_list, start=1):
        print(f"\t{idx}. {c.choice_text} - {c.votes}")

1. 좋아하는 색깔은 무엇입니까?
	1. 파란색 - 7
	2. 노란색 - 0
	3. 주황색 - 0
	4. 보라색 - 0
	5. 빨간색 - 3
	6. 보라색 - 5
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔을 고르세요
4. 여행가고 싶은 나라는?


# insert / update
- `모델객체.save()`
- 모델객체의 pk가 DB에 없으면 insert, 있으면 update

In [23]:
# insert
## question: pk_id : 자동증가 정수(생략), pub_date: insert할 때 일시를 추가(생략)
new_q = Question(question_text="배우고 싶은 언어는 무엇입니까?")
print(new_q.question_text)
print(new_q.pk, new_q.pub_date)

배우고 싶은 언어는 무엇입니까?
None None


In [24]:
# 저장 - pk: None (DB에 없는 데이터) ==> insert
new_q.save()

In [25]:
# insert 후에 자동저장되는 값들(pk, pub_date)이 모델객체의 field에 저장된다.
print(new_q.pk)
print(new_q.pub_date)

5
2025-07-08 02:23:45.382728+00:00


In [40]:
q = Question.objects.get(pk=4)
q.question_text = "여행으로 가고 싶은 나라를 선택해주세요."

In [41]:
q

<Question: 4. 여행으로 가고 싶은 나라를 선택해주세요.>

In [36]:
# q.pk = 4 -> DB에 있는 pk => update
q.save()

In [42]:
for q in Question.objects.all():
    print(q)

1. 좋아하는 색깔은 무엇입니까?
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔을 고르세요
4. 여행으로 가고 싶은 나라를 선택해주세요.
5. 배우고 싶은 언어는 무엇입니까?


In [48]:
q = Question.objects.get(pk=4)

In [50]:
q.choice_set.all()

<QuerySet []>

In [52]:
# FK field -> 부모 Model 객체 (참조 Field값만 instance변수로 가지면 된다.)
c = Choice(choice_text="미국", question=q)
c.save()

In [53]:
q.choice_set.all()

<QuerySet [<Choice: 7. 미국>]>

In [54]:
c = Choice(choice_text="일본", question=q)
c.save()

In [55]:
q.choice_set.all()

<QuerySet [<Choice: 7. 미국>, <Choice: 8. 일본>]>

In [56]:
for c in q.choice_set.all():
    print(c.pk, c.choice_text, c.votes)

7 미국 0
8 일본 0


In [60]:
qq = Question(pk=4)
c = Choice(choice_text="영국", votes=20, question=qq)
c.save()

In [61]:
for c in q.choice_set.all():
    print(c.pk, c.choice_text, c.votes)

7 미국 0
8 일본 0
9 영국 20


# delete
- `모델객체.delete()`
    - 모델객체의 pk의 data를 DB에 삭제.

In [62]:
c15 = Choice.objects.get(pk=3)
c15.delete()

(1, {'polls.Choice': 1})

In [67]:
# choice에서 id가 10이상인 값들을 삭제
# delete from choice where id >= 10
del_c = Choice.objects.filter(pk__gte=7)
for c in del_c:
    c.delete()

In [68]:
for c in Choice.objects.all():
    print(c)

1. 파란색
2. 노란색
4. 보라색
5. 빨간색
6. 보라색


In [72]:
result = Choice.objects.raw("select * from polls_choice")
for r in result:
    print(r, r.question)

1. 파란색 1. 좋아하는 색깔은 무엇입니까?
2. 노란색 1. 좋아하는 색깔은 무엇입니까?
4. 보라색 1. 좋아하는 색깔은 무엇입니까?
5. 빨간색 1. 좋아하는 색깔은 무엇입니까?
6. 보라색 1. 좋아하는 색깔은 무엇입니까?


In [ ]:
def solution(numbers):
    binary = [list('0'+bin(num)[2:]) for num in numbers]
    for token in binary:
        if token[len(token)]==0:
            token[len(token)-1]=1
        elif token[len(token)]==1 & token[len(token)-1]==0:
            token[len(token)]=0
            token[len(token)-1]=1
        elif token
        
    # return binary

numbers = [3, 5, 8, 0]
print(solution(numbers))

[['0', '1', '1'], ['0', '1', '0', '1'], ['0', '1', '0', '0', '0'], ['0', '0']]


In [122]:
def solution(numbers):
    binary = [list('0'+bin(num)[2:])[::-1] for num in numbers]
    return binary

numbers = [3, 5, 8, 0]
type(solution(numbers))

list

In [145]:
binary = [list('0'+bin(num)[2:])[::-1] for num in numbers]
for i in range(len(binary)):
    if binary[i][0]==0:
        binary[i][0]=1
    elif binary[i][0]==1 and binary[i][1]==0:
        binary[i][0], binary[i][1] = binary[i][1], binary[i][0]
print(binary)

[['1', '1', '0'], ['1', '0', '1', '0'], ['0', '0', '0', '1', '0'], ['0', '0']]


In [ ]:
def solution(numbers):
    answer = []
    binary = [list('0'+bin(num)[2:])[::-1] for num in numbers]
    for i in range(len(binary)):
        for j in range(len(binary[i])-1):
            if binary[i][0]=='0':
                binary[i][0]='1'
                break
            elif binary[i][j]=='1' and binary[i][j+1]=='0':
                binary[i][j], binary[i][j+1] = binary[i][j+1], binary[i][j]
                break
        decimal = int(''.join(binary[i][::-1]), 2)
        answer.append(decimal)
    return answer

[5, 6, 11, 9, 11]


In [ ]:
def solution(bridge_length, weight, truck_weights):
    time = 0
    sum_w = 0
    cnt = 1

    if len(truck_weights)==1:
        time += bridge_length
    while sum_w > weight:
        for n in range(len(truck_weights)):
            sum_w += truck_weights[n]
            cnt += 1
    time += bridge_length-(bridge_length-cnt)
    
    answer = 1 + time
    return answer

a = 100, b= 100, c = [10]


In [9]:
def solution(bridge_length, weight, truck_weights):
    time = 0
    sum_w = 0
    cnt = 1
    
    if len(truck_weights)==1:
        time += bridge_length
    for w in range(len(truck_weights)-1):
        while 1:
            if sum_w <= weight:
                sum_w += truck_weights[w]
                cnt += 1
            cnt=0
            break
        time += bridge_length+cnt-1
    answer = 1 + time
    return answer

a=2
b=10
c=[7,4,5,6]
solution(a,b,c)

4

In [ ]:
q = Question.objects.get(pk=1)
print(q.pk, q.question_text)
for c in q.choice_set.all():
    print(c.pk, c.choice_text)

In [1]:
def solution(ingredient):
    ingredient = " ".join(map(str, ingredient))
    target = '1231'
    cnt = 0
    i = 0
    while i<len(ingredient):
        text = ''
        if ingredient[i:i+len(target)] == target:
            cnt += 1
            i += len(target)
        else :
            text += ingredient[i]
            i += 1
    answer = cnt
    return answer

In [ ]:
ingredient = [2,1,1,2,3,1,2,3,1]
target = [1,2,3,1]
cnt = 0
i = 0
text = ''
text_len = len(ingredient)
while 1:
    while i<text_len:
        if text[i:i+len(target)] == target:
            cnt += 1
            i += len(target)
        else :
            text.append(text[i])
            i += 1
    print(text)
    if target in text:s
        print(text)
    else: break
        
cnt

AttributeError: 'str' object has no attribute 'append'